Мне нужно защитить данные клиентов страховой компании «Хоть потоп». Необходимо разработайте такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Также необходимо обосновать корректность его работы.



## 1. Загрузка данных

In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import r2_score
from sklearn.linear_model import LinearRegression

In [2]:
data = pd.read_csv('/datasets/insurance.csv')

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
Пол                  5000 non-null int64
Возраст              5000 non-null float64
Зарплата             5000 non-null float64
Члены семьи          5000 non-null int64
Страховые выплаты    5000 non-null int64
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


In [4]:
data.head(10)

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0
5,1,43.0,41000.0,2,1
6,1,39.0,39700.0,2,0
7,1,25.0,38600.0,4,0
8,1,36.0,49700.0,1,0
9,1,32.0,51700.0,1,0


Мы видим, что у столбцов Возраст и Зарплата можно изменить тип на целочисленный. 

In [5]:
#Изменим тип столбцов Возраст и Зарплата на целочисленный
data['Возраст'] = data['Возраст'].astype('int')
data['Зарплата'] = data['Зарплата'].astype('int')

In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
Пол                  5000 non-null int64
Возраст              5000 non-null int64
Зарплата             5000 non-null int64
Члены семьи          5000 non-null int64
Страховые выплаты    5000 non-null int64
dtypes: int64(5)
memory usage: 195.4 KB


## 2. Умножение матриц

В этом задании вы можете записывать формулы в *Jupyter Notebook.*

Чтобы записать формулу внутри текста, окружите её символами доллара \\$; если снаружи —  двойными символами \\$\\$. Эти формулы записываются на языке вёрстки *LaTeX.* 

Для примера мы записали формулы линейной регрессии. Можете их скопировать и отредактировать, чтобы решить задачу.

Работать в *LaTeX* необязательно.

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

Так как мы получили одинаковую точность на оригинальной модели и на преобразованных фичах, то мы можем предположить что:
$$
a = X \cdot w = X\cdot P \cdot w \Rightarrow
$$
$$
X \cdot w = X \cdot P \cdot w \Rightarrow
$$
$$
X\cdot(X^T \cdot X)^{-1} \cdot X^T \cdot y = X \cdot P \cdot ((X \cdot P)^T \cdot X \cdot P)^{-1} \cdot (X \cdot P)^T \cdot y \Rightarrow
$$
$$
X\cdot(X^T \cdot X)^{-1} \cdot X^T \cdot y = X \cdot P \cdot (P^T \cdot X^T \cdot X \cdot P)^{-1} \cdot P^T \cdot X^T \cdot y  \Rightarrow
$$
$$
X\cdot(X^T \cdot X)^{-1} \cdot X^T \cdot y = X \cdot P \cdot P^{-1} \cdot X^{-1} \cdot (X^T)^{-1} \cdot (P^T)^{-1} \cdot P^T \cdot X^T \cdot y\Rightarrow
$$
$$
X\cdot(X^T \cdot X)^{-1} \cdot X^T \cdot y = X \cdot E \cdot (X^T \cdot X)^{-1} \cdot E \cdot X^T \cdot y  \Rightarrow
$$
$$
X\cdot(X^T \cdot X)^{-1} \cdot X^T \cdot y = X\cdot(X^T \cdot X)^{-1} \cdot X^T \cdot y
$$

Обе стороны равны, следовательно качество модели при умножении фичей на нормальную матрицу, качество не должно меняться

##### 2.1. Разделение датасета на фичи и таргеты.

In [7]:
#Поделим наш датафрейм
features = data.drop('Страховые выплаты', axis=1)
target = data['Страховые выплаты']


In [8]:
#Создадим матрицу с нормальным распределением и проверим её на обратимость
state = np.random.RandomState(12345)
transform_matrix = state.normal(size=(features.shape[1], features.shape[1]))
rev_matrix = np.linalg.inv(transform_matrix)

Мы создали обратиму матрицу со случайными значениями с нормальным распределением.

In [9]:
#Проверим качечество предсказания модели на оригинальных данных
model = LinearRegression()
model.fit(features, target)
predictions = model.predict(features)
print('R2-score на выборке без преобразований',r2_score(target, predictions))



R2-score на выборке без преобразований 0.42494550308169177


In [10]:
multiplied_features = features @ transform_matrix

In [11]:
model2 = LinearRegression()
model2.fit(multiplied_features, target)
predictions2 = model2.predict(multiplied_features)
print('R2-score на выборке с преобразованием:',r2_score(target, predictions2))


R2-score на выборке с преобразованием: 0.42494550308169066


**Ответ:** Значение метрики не изменилось

**Обоснование:** Так как сгенерированная матрица имеет нормальное распределение, то изменение значений в оригинальных данных будет происходить примерно на одинаковые значения, что не должно влият на качество нашей модели.

## 3. Алгоритм преобразования

**Алгоритм**

Для шифровки персональных данных мы можем сгенерировать случайную обратимую матрицу с нормальным распределением. Далее умножить матрицу признаков на созданную матрицу (при этом значение метрики не изменится). Далее для расшифровки модели мы можем умножить преобразованные признаки на обратную сгенерированную матрицу

**Обоснование**

$$
Ax = b \Rightarrow
$$

$$
x = A^{-1}b 
$$

## 4. Проверка алгоритма

In [12]:
model = LinearRegression()
model.fit(features, target)
predictions = model.predict(features)
print('R2-score на выборке без преобразований',r2_score(target, predictions))

R2-score на выборке без преобразований 0.42494550308169177


In [13]:
uncracked_features = multiplied_features @ rev_matrix
#Проверим работу алгоритма
uncracked_features.head()

,0,1,2,3
0,1.000000e+00,41.0,49600.0,1.000000e+00
1,-6.449658e-12,46.0,38000.0,1.000000e+00
2,1.576164e-12,29.0,21000.0,-1.828262e-12
3,-9.028180e-12,21.0,41700.0,2.000000e+00
4,1.000000e+00,28.0,26100.0,-1.589574e-12


Вывод: Мы получили изначальные признаки. Проверим изменилось ли качество модели

In [14]:
model = LinearRegression()
model.fit(uncracked_features, target)
predictions = model.predict(uncracked_features)
print('R2-score на выборке без преобразований',r2_score(target, predictions))

R2-score на выборке без преобразований 0.424945503081693


<strong>Вывод:</strong> Качество модели не изменилось. Мы выполнили матричные операции для преобразования исходных данных. Так как мы генерировали случайную матрицу, то изначальные данные изменились на случайную величину по нормальному закону. Качество модели при этом не изменилось. Для расшифровки данных, шифрованные значения необходимо умножить на обратную сгенерированную матрицу. Качество модели, при этом также не изменится  